In [ ]:
#Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip uninstall spacy

Found existing installation: spacy 2.2.4
Uninstalling spacy-2.2.4:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/bin/*
    /usr/local/lib/python3.7/dist-packages/spacy-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/bin/theano_cache.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.2.4


In [ ]:
!pip install transformers
!pip install seqeval
!pip install emoji
!pip install spacy
!python -m spacy download ca_core_news_sm
!python -m spacy download ca_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 6.6 MB 41.6 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires spacy>=2.0.18; python_version < "3.8", which is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=594a001735a385454326

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.6 MB 3.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.2 MB 65.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_md')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification, BertTokenizer, BertForSequenceClassification
from seqeval.metrics import classification_report
import pickle
import nltk
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from nltk import word_tokenize
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import emoji
def give_emoji_free_text(text):
    return emoji.replace_emoji(text, replace='')

In [ ]:
path = '/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Data/Train Dataset/catalan_named_entity.xlsx'
all_dataset = pd.read_excel(path)

In [ ]:
all_dataset.head()

,username,creation_date,tweet,likes,retweets,crime_label,crime_type,location
0,mossos+465:40,2017-06-14 09:02:20,Robaven en empreses i bars de les comarques ta...,9,1,Crime,Theft,NaN
1,mossos,2016-06-07 18:56:05,"Circula begut, sota els efectes de drogues i s...",6,11,Crime,Public Disorder,Sitges
2,mossos,2013-09-29 09:28:44,Ingressa a presó l’autor de 5 robatoris violen...,0,4,Crime,Robbery,"Granollers,Cardedeu,Canovelles"
3,mossos,2021-04-21 07:58:44,Desmantellem amb @GUBBarcelona un punt de vend...,206,23,Crime,Drug Trafficking,"barri del Raval,BCN"
4,mossos,2019-09-13 17:04:00,Detenim tres persones per robar cablejat de co...,107,16,Crime,Theft,"Salou,Sant Vicenç de Calders"


Spacy

In [ ]:
import ca_core_news_sm
import ca_core_news_md

nlp_1 = ca_core_news_sm.load()
nlp_2 = ca_core_news_md.load()

In [ ]:
all_dataset['loc_spacy'] = ''
for i in range(len(all_dataset)):
  document = nlp_1(give_emoji_free_text(all_dataset.tweet[i]))
  loc = []
  for named_entity in document.ents:
    if named_entity.label_ == 'LOC':
      loc.append(str(named_entity))
  all_dataset['loc_spacy'][i] = ",".join(loc)

In [ ]:
all_dataset['loc_spacy_1'] = ''

for i in range(len(all_dataset)):
  document = nlp_2(give_emoji_free_text(all_dataset.tweet[i]))
  loc = []
  for named_entity in document.ents:
    if named_entity.label_ == 'LOC':
      loc.append(str(named_entity))
  all_dataset['loc_spacy_1'][i] = ",".join(loc)

BERT

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, FillMaskPipeline
from transformers import pipeline


tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-cased-ner")

model = AutoModelForTokenClassification.from_pretrained("projecte-aina/roberta-base-ca-cased-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/863k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/517k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [ ]:
def ber_ner(index):
  sentence = give_emoji_free_text(all_dataset.tweet[index])
  ner_results = nlp(sentence)
  doc = {'text': sentence, 'ents': []}
  count=0
  same_entity = False
  previous_entity = ''
  last_start = 0
  for i in range(len(ner_results)): 
    if 'LOC' not in ner_results[i]['entity']:
      continue
    else: 
      if ner_results[i]['entity'].startswith('B'): 
        doc['ents'].append({'word':sentence[ner_results[i]['start']: ner_results[i]['end']] ,'label': ner_results[i]['entity'].replace('B-', '')})
        same_entity = True
        previous_entity = ner_results[i]['entity'].replace('B-', '')
        last_index =  ner_results[i]['start']
      elif ner_results[i]['entity'].startswith('I'): 
        if ner_results[i]['entity'].replace('I-', '') != previous_entity:
          print(ner_results[i]['entity'].replace('I_', ''))
          same_entity = False
        if same_entity:
          doc['ents'][-1] = {'word':sentence[last_index: ner_results[i]['end']], 'label': previous_entity}
  return doc['ents']

In [ ]:
all_dataset['loc_bert'] = ''
for i in range(len(all_dataset)):
    pred = ber_ner(i)
    loc = []
    for j in range(len(pred)):
      if pred[j]['word'] != '#':
        loc.append(pred[j]['word'])
    all_dataset['loc_bert'][i] = ",".join(loc)

In [ ]:
from spacy import displacy

def display(sentence, ner_results):
  doc = {'text': sentence, 'ents': []}
  count=0
  same_entity = False
  previous_entity = ''
  last_start = 0
  for i in range(len(ner_results)): 
    if 'MISC' in ner_results[i]['entity']:
      continue
    else: 
      if ner_results[i]['entity'].startswith('B'): 
        doc['ents'].append({'start':ner_results[i]['start'], 'end': ner_results[i]['end'], 'label': ner_results[i]['entity'].replace('B_', '')})
        same_entity = True
        previous_entity = ner_results[i]['entity'].replace('B_', '')
        last_start = ner_results[i]['start']
      elif ner_results[i]['entity'].startswith('I'): 
        if ner_results[i]['entity'].replace('I_', '') != previous_entity:
          same_entity = False
        if same_entity:
          doc['ents'][-1] = {'end':ner_results[i]['end'], 'label': previous_entity, 'start':last_start}

  tags = set(['LOC','MISC','ORG','PER'])
  colors = ["#7aecec", "#bfeeb7", "#feca74", "#ff9561", "#aa9cfc", "#c887fb", "#9cc9cc", "#ffeb80", "#ff8197", "#ff8197", "#f0d0ff", "#bfe1d9", "#bfe1d9", "#e4e7d2", "#e4e7d2", "#e4e7d2", "#e4e7d2", "#e4e7d2"]
  tags_colors = {tag: color for tag, color in zip(tags, colors)}
  options = {'ents': tags, 'colors': tags_colors}

  displacy.render(doc, style="ent", manual=True, jupyter=True, options=options)


In [ ]:
sentence = all_dataset.tweet[2]
ner_results = nlp(sentence)
display(sentence, ner_results)

In [ ]:
all_dataset.loc[all_dataset.loc[all_dataset.location.isnull()].index,'location'] = ''

In [ ]:
def precision_recall(file, column):
  true_positive = 0
  false_positive = 0
  false_negative = 0
  for i in range(len(file)):
    loc = list(file.location[i].split(','))
    #print(loc)
    pred = list(file[column][i].split(','))
    #print(pred)
    for i in range(len(loc)):
      for j in range(len(pred)):
        if pred[j] == loc[i]:
          true_positive += 1
          #print(pred[j],loc[i])
    for j in range(len(pred)):
      if  pred[j] not in loc: 
        false_positive += 1
        #print(pred[j])
    for j in range(len(loc)):
      if  loc[j] not in pred: 
        false_negative += 1
        #print(loc[j])
  precision = true_positive / (true_positive + false_positive)
  recall = true_positive / (true_positive + false_negative)
  return precision, recall

In [ ]:
precision_bert, recall_bert = precision_recall(all_dataset, 'loc_bert')
print('BERT')
print('Precision:', precision_bert)
print('Recall:', recall_bert)
print('F1:', (2*precision_bert*recall_bert)/(precision_bert+recall_bert))

BERT
Precision: 0.7671232876712328
Recall: 0.8115942028985508
F1: 0.7887323943661971


In [ ]:
precision_spacy, recall_spacy = precision_recall(all_dataset, 'loc_spacy')
print('SPACY MD')
print('Precision:', precision_spacy)
print('Recall:', recall_spacy)
print('F1:', (2*precision_spacy*recall_spacy)/(precision_spacy+recall_spacy))


SPACY MD
Precision: 0.5753424657534246
Recall: 0.6086956521739131
F1: 0.5915492957746479


In [ ]:
precision_spacy_1, recall_spacy_1 = precision_recall(all_dataset, 'loc_spacy_1')
print('SPACY SM')
print('Precision:', precision_spacy_1)
print('Recall:', recall_spacy_1)
print('F1:', (2*precision_spacy_1*recall_spacy_1)/(precision_spacy_1+recall_spacy_1))


SPACY SM
Precision: 0.6285714285714286
Recall: 0.6376811594202898
F1: 0.6330935251798562
